<br>

---

## 1.0 - Importing Modules

---
<br>

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
import tensorflow as tf
print("Importing tensorflow")

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

Importing tensorflow


<br>

---

## 2.0 - Loading Models

---
<br>

### Load Model (Not Fine Tuned)

In [2]:
model = keras.models.load_model('trained_models/um_combined_model.h5')

### Load Model (Fine Tuned)

In [2]:
model = keras.models.load_model('trained_models/um_combined_tuned_model.h5')

In [3]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mobilenetv2_1.00_224 (Functi (None, 10, 10, 1280)      2257984   
_________________________________________________________________
global_average_pooling2d (Gl (None, 1280)              0         
_________________________________________________________________
dropout (Dropout)            (None, 1280)              0         
_________________________________________________________________
dense (Dense)                (None, 256)               327936    
_________________________________________________________________
dense_1 (Dense)              (None, 4)                 1028      
Total params: 2,586,948
Trainable params: 2,190,404
Non-trainable params: 396,544
_________________________________________________________________


<br>

---

## 3.0 - Prediction

---
<br>

The probabilities are __[BrownSpot, Healthy, Hispa, LeafBlast]__ respectively

In [11]:
# Import basic GUI functionality
from ipywidgets import Button
from tkinter import Tk, filedialog
from IPython.display import clear_output, display

In [12]:
def select_files(b):
    clear_output()
    
    root = Tk()
    root.withdraw() # Hide the main window.
    root.call('wm', 'attributes', '.', '-topmost', True) # Raise the root to the top of all windows.
    b.files = filedialog.askopenfilename(multiple=True) # List of selected files will be set button's file attribute.

    print(f'Loading the image: {b.files[0]}')
    
    img = keras.preprocessing.image.load_img(
        b.files[0], target_size=(300,300)
    )
    
    plt.figure()
    plt.imshow(img)
    
    img_array = keras.preprocessing.image.img_to_array(img)
    # Map to [-1, 1] range
    img_array = (img_array / 127.5) - 1
    img_array = tf.expand_dims(img_array, 0) # Create a batch
    
    # [BrownSpot, Healthy, Hispa, LeafBlast]
    predicted_prob = model.predict(img_array)
    print(predicted_prob * 100)

In [20]:
# Register the event listener and display the button 
fileselect = Button(description="File select")
fileselect.on_click(select_files)

#[BrownSpot, Healthy, Hispa, LeafBlast]
display(fileselect)

IndexError: string index out of range

<br>

---

## 4.0 - Evaluate Test Dataset

---
<br>

In [8]:
IMG_SIZE = (300, 300)

In [9]:
# Validation Dataset
VALIDATE_DIR = 'datasets/RiceDiseaseDataset/validation/Combined'

validate_ds = tf.keras.preprocessing.image_dataset_from_directory(
    VALIDATE_DIR,
    seed=123,
    image_size=IMG_SIZE,
    label_mode='categorical'
)

# Scale to [-1, 1]
validate_ds = validate_ds.map(lambda x,y: (keras.layers.experimental.preprocessing.Rescaling(1./127.5, offset=-1)(x), y) )

Found 492 files belonging to 4 classes.


In [10]:
# Validate the model
model.evaluate(validate_ds)

16/16 [==============================] - 40s 508ms/step - loss: 1.3055 - accuracy: 0.6768


[1.305471420288086, 0.6768292784690857]

<br>

---

## 5.0 - Convert to Tensorflow.js Compatible

---
<br>

In [10]:
!pip install tensorflowjs

Microsoft Windows [Version 10.0.19042.1110]
(c) Microsoft Corporation. All rights reserved.

(venv) D:\My Desktop\Programming\Projects\MachineLearning>pip install tensorflowjs

(venv) D:\My Desktop\Programming\Projects\MachineLearning>

In [4]:
import tensorflowjs as tfjs

In [5]:
!tensorflowjs_converter --input_format=keras --output_format=tfjs_layers_model trained_models/um_combined_model.h5 trained_models/TensorflowJS/um_combined_model 

2021-08-06 03:59:46.479073: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library cudart64_110.dll
